# **Poker Tournament scheduler**

### *Main Idea:*
Poker proffesionals effectively run a business in which their decision making skills are the product, and applied over time will allow them to be proffitable. Many people are actively using data science and game theory to get better at playing the game, and increasing profit based on their decision in the game. However, one of the most important decisions comes before you ever sit down to play, deciding which game to play in. Cash games are more difficult to obtain any data on because, as the name implies, the games operate in cash, and aren't easily tracked. Poker tournaments however will be completely tracked, and lots of data will be available for them.

With all of this in mind, the following objective is to find all the information that is available on poker tournaments around the world, and create a dashboard that can be interactively filtered so that an end user can develop a poker tournamnet scheduler for themeselves.

### *Steps*

1. Find and scrape as much data as I can on poker tournaments
2. Clean all available data
3. Brainstorm ways to classify tournaments as high or low value, and sort tournaments for end user filtering
4. Bring data into Tableau and create a dashboard that is easily navigable
5. Allow end users to export a CSV of their tournament schedule for the next 365 days

### *Set up imports*

In [249]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### *Create CSV to store data*

In [250]:
file = open("Tournaments_data_3.csv", "w")
writer = csv.writer(file)
writer.writerow(["Unique ID", "Date", "Start time", "location", "Venue", "Game type", "Total buy in", "Prize pool", "rake", "Guarantee"])

91

### set up webdriver

In [251]:
browser_driver = Service('/Users/alexgiovannini/Desktop/DevCC/Week 19 - 23 capstone/Project ideas/Poker/chromedriver')
scraper = webdriver.Chrome(service=browser_driver)
wait = WebDriverWait(scraper, 10)

### *Get location elements from "https://www.pokeratlas.com/areas" and store them as text in "locations_text_list." Then set up the unique ID and "locations_scraped" list to be used later*

In [252]:
scraper.get("https://www.pokeratlas.com/areas")

try:
    popup = scraper.find_element(By.CLASS_NAME, "modal-close")
    wait.until(EC.element_to_be_clickable(popup))
    popup.click()
except NoSuchElementException:
    pass
except StaleElementReferenceException:
    pass

try:
    cookies = scraper.find_element(By.XPATH, "//*[@id='modal-gdpr-cookies']/button")
    cookies.click()
except NoSuchElementException:
    pass
except StaleElementReferenceException:
    pass

US_element = scraper.find_element(By.CLASS_NAME, "country")
locations_list = US_element.find_elements(By.TAG_NAME, "li")
locations_text_list = []
for location in locations_list:
    locations_text_list.append(location.text)
unique_id = 0
locations_scraped = []

# Pokeratlas.com scraper version 1.
### Run this script to scrape tournaments using link text iterating through "locations_text_list"

In [283]:

scraper.get("https://www.pokeratlas.com/areas")
file = open("Tournaments_data_5.csv", "w")
writer = csv.writer(file)
writer.writerow(["Unique ID", "Date", "Start time", "location", 
"Venue", "Game type", "Total buy in", "Prize pool", "rake", "Guarantee"])
for location in locations_text_list:
    try:
        popup = scraper.find_element(By.CLASS_NAME, "modal-close")
        wait.until(EC.element_to_be_clickable(popup))
        popup.click()
    except TimeoutException:
        pass
    except NoSuchElementException:
        pass
    except StaleElementReferenceException:
        pass

    if location not in locations_scraped:
        scraper.find_element(By.LINK_TEXT, location).click()
        locations_scraped.append(location)

        time.sleep(2)
        try:
            popup = scraper.find_element(By.CLASS_NAME, "modal-close")
            wait.until(EC.element_to_be_clickable(popup))
            popup.click()
        except TimeoutException:
            pass
        except NoSuchElementException:
            pass
        except StaleElementReferenceException:
            pass

        time.sleep(1)
        try:
            see_all_tournaments = scraper.find_element(By.LINK_TEXT, 'See All Tournaments')
            see_all_tournaments.click()
        except NoSuchElementException:
            scraper.back()
            continue

        time.sleep(2)
        try:
            popup = scraper.find_element(By.CLASS_NAME, "modal-close")
            popup.click()
        except ElementNotInteractableException:
            pass
        except NoSuchElementException:
            pass
        except StaleElementReferenceException:
            pass

        scroll_pause_time = 1

        screen_height = scraper.execute_script("return window.screen.height;")
        i = 1

        while True:
            scraper.execute_script("window.scrollTo(0, {screen_height} * {i});".format(screen_height=screen_height, i=i))
            i += 1
            time.sleep(scroll_pause_time)
            scroll_height = scraper.execute_script("return document.body.scrollHeight;")
            if (screen_height * i > scroll_height):
                break

        tournaments = scraper.find_elements(By.CLASS_NAME, "tournament-item")

        for tournament in tournaments:
            time.sleep(2)
            if ("warning" in tournament.get_attribute("class")):
                continue
            else:
                venue = tournament.find_element(By.CLASS_NAME, "venue-name").text
                date = tournament.find_element(By.CLASS_NAME, "date").text
                start_time = tournament.find_element(By.CLASS_NAME, "time").text
                game_type = tournament.find_element(By.CLASS_NAME, "type").find_element(By.TAG_NAME, "span").text
                guarantee = tournament.find_element(By.CLASS_NAME, "name").text
                tournament.click()
                time.sleep(1)
                buy_in_details = scraper.find_element(By.CLASS_NAME, "buy-in-details")
                buy_in_list = buy_in_details.find_elements(By.CSS_SELECTOR, "dd")
                if len(buy_in_list) < 3:
                    total_buy_in = "day 2"
                else:
                    total_buy_in = buy_in_list[0].text
                    prize_pool = buy_in_list[1].text
                    rake = buy_in_list[2].text
                scraper.back()
                writer.writerow([unique_id, date, start_time, location, venue, game_type, total_buy_in, prize_pool, rake, guarantee])
                unique_id += 1
        scraper.back()
        scraper.back()

file.close()

The script ran for 319 minutes before I restarted it. it seemed to have hit the scraper.back() to many times and ended up at the start page.

It got stuck at line 3917.

The writer isn't writing because the file is closed. I am going to open a new file, "tournaments_data_4" and continue writing from there.

Everytime I ran the script, it added a new location to the "locations_scraped", and then attempted to scrape it. once the error occured and none had been scraped, I had to ".pop()" 7 times in order to return to the next location. because of this, I may be missing some data from  "North and South Dakota" past 2/10.

The script ran for an hour before running into an error. I also didn't update it to open 4 instead of three. That one hour was only las vegas. So i then popped las vegas out and will re run again.

I got another error on 3/3 in las vegas, So any tournaments past march 3 in vegas are missing.

I also never reset the unique ID after running it a few times, so I will need to manually adjust the unique ID's when I combine the various CSV files.

The script has now finished. I will most likely create a version 2.0 that has the following features:
- Is faster
- Scrapes Canada as well
- Scrapes other countries if possible
- automatically restarts if encounters an error. If it needs to restart, it will need to pop the last "locations_scraped" to get that data again
